# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [ ]:
# dependencies
!pip install --upgrade boto3
!pip install langchain
!pip install langchainhub
!pip install anthropic

## check bedrock availability

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock
from utils.GamePlayer import GamePlayer
from utils.GameAssistant import GameAssistant
from utils.GameMaster import GameMaster
from utils.PeTemplates import *
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error

boto3_bedrock = bedrock.get_bedrock_client()

# print_ww("hello")
# Info("hello")

GA = GameAssistant(template_assistant_role, 1000)
print(GA.agent.prompt.template)

## Test Part

In [ ]:
from collections import Counter

def find_most_frequent(arr):
  freq = Counter(arr)
  max_count = max(freq.values())
  
  return [k for k, v in freq.items() if v == max_count]

vote_names = ['p1','p1','p2','p2','p3','p5','p4','p4']
AA = Counter(vote_names)
print(AA.most_common())

print(find_most_frequent(vote_names))

aa = []

if len(aa) != 1:
    print(1)

In [2]:
from utils.GamePlayer import GamePlayer
from utils.GameMaster import GameMaster
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error

GM = GameMaster(10, True)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

02/18/2024 04:16:00 PM 	===== P1:存活,P2:淘汰,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 ResetGame =====
02/18/2024 04:16:00 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:00 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:00 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:01 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:01 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:01 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:01 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:01 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:01 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/18/2024 04:16:01 PM Found credentials from IAM Role: BaseNotebookInstanceEc

True
 根据你提供的信息,我判断现在是第一天的夜晚阶段,身为狼人的我需要与另一名狼人P6进行商讨并决定今晚消灭的目标。

为了隐藏身份,我建议行动如下:

{"action": "WolfVote", "target": "P2"}

我随机选择了一个玩家P2进行投票。这可以避免暴露我和P6的狼人身份。在第一天的时候随机投票是一个比较安全的做法。

02/18/2024 04:16:12 PM 		===== DoAction NIGHT 1 P1 ======
02/18/2024 04:16:12 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P1', 'status': 1, 'role': '狼人', 'character': '话痨', 'response': ['{"action": "WolfVote", "target": "P2"}']}

02/18/2024 04:16:12 PM 		===== DoMemory NIGHT 1 P6 ======


 狼人杀游戏第1晚,P1作为狼人,投票杀死了P2。

02/18/2024 04:16:13 PM 		===== DoAnswer NIGHT 1 P6 ======
02/18/2024 04:16:13 PM 	QUESTION: 现在是NIGHT 1,你是玩家P6(狼人身份,本阵营为:P1,P6).目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活(逗号为分割符).现在是第1天夜晚，该如何行动?


...

02/18/2024 04:32:59 PM 		===== DoAnswer DAY 3 P7 ======
02/18/2024 04:32:59 PM 	QUESTION: 现在是DAY 3,你是玩家P7(村民身份)..目前场上玩家:P1:存活,P2:淘汰,P3:淘汰,P4:存活,P5:淘汰,P6:存活,P7:淘汰,P8:存活(逗号为分割符).现在是第3天白天，你已经死亡,有何遗言?


True
 根据你提供的信息,我作为已死亡的村民P7,在DAY 3应该留下遗言。

考虑到我在游戏中一直保持中立谨慎的立场,我的遗言应该体现这一点,同时尽可能提供有价值的信息帮助村民阵营。但是也不能轻易相信或否定任何一方。

我的遗言建议是:

{"action": "DeathWords", "content": "我在游戏中一直保持中立。近期P1和P6的举动比较可疑,但也还需要进一步证据。希望大家不要被任何一方迷惑,继续保持理性观察。我相信时间最终会给出答案。"}

这样的遗言既体现了我谨慎的性格,也提供了某种信息帮助村民阵营,但又不会绝对站队任何一方。希望这对村民赢得游戏有所帮助。请让我知道如果需要进一步解释或修改。

02/18/2024 04:33:22 PM 		===== DoAction DAY 3 P7 ======
02/18/2024 04:33:22 PM 
 [ROUND ACTION]={'time': 'DAY 3', 'player': 'P7', 'status': 0, 'role': '村民', 'character': '观察家', 'response': ['{"action": "DeathWords", "content": "我在游戏中一直保持中立。近期P1和P6的举动比较可疑,但也还需要进一步证据。希望大家不要被任何一方迷惑,继续保持理性观察。我相信时间最终会给出答案。"}']}

02/18/2024 04:33:22 PM 	时间DAY 3,场上存活状态 狼人:2 村民:2
02/18/2024 04:33:22 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


 根据游戏过程,我对这个游戏进行了简要总结:

第一天晚上,狼人P1和P6投票淘汰了村民P2。 

第二天白天,P1反对相信P2死前的话,并投票淘汰了村民P3。P5投票淘汰P1。本轮投票淘汰村民P3。

第二天晚上,狼人P1和P6投票淘汰了村民P5。

第三天白天,P5死前遗言怀疑P1是狼人。P4和P7、P8提出继续观察,不要下定论。P1投票淘汰P4,P6投票淘汰P7,投票未通过。最终P1和P6联合投票淘汰了村民P7。

根据游戏结果,可以看出两名狼人P1和P6成功淘汰了全部村民,狼人获得胜利。P1和P6在每晚和白天的投票中保持一致,且多次淘汰村民。P5和P7死前都对P1产生了怀疑。

以上内容遵循客观描述游戏过程的原则,不包含任何主观推理或意见,也过滤掉了无关内容,重点突出了狼人和村民的淘汰情况以及P1和P6的联合投票行为。请检查是否符合要求,在字数限制内客观概述了游戏的主要过程和结果。

02/18/2024 04:33:52 PM 狼人胜利!
02/18/2024 04:33:52 PM 游戏结束.
02/18/2024 04:33:52 PM ===== P1:存活,P2:淘汰,P3:淘汰,P4:存活,P5:淘汰,P6:存活,P7:淘汰,P8:存活 EndGame =====
02/18/2024 04:33:52 PM 	===== input_tokens: 597048 output_tokens 18024 ======
02/18/2024 04:33:52 PM 	===== elapsed_time: 1071.3879146575928 ======
